# Overview

**Oftentimes, we want to test some LLMs that are not widely available or are expensive to host. There are various ways to still get your hands dirty with them.**


**In this notebook, I'm showing how to use LangChain's CustomLLM together with <a href="https://openrouter.ai/">OpenRouter.ai</a>'s collection of LLMs (like gpt-4-32k, claude-2, and many more) to use these models in your LangChain apps.**

### <font color='gray'>Loading libraries and variables</font>

In [3]:
import os
import requests, json
from dotenv import load_dotenv
import re
import zipfile
import io
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from typing import Any, List, Mapping, Optional
from langchain.callbacks.manager import CallbackManagerForLLMRun
from langchain.llms.base import LLM
from langchain_core.output_parsers import StrOutputParser

output_parser = StrOutputParser()
load_dotenv()

True

In [4]:
API_KEY = os.getenv('OPENROUTER_API_KEY')

MODEL_CODELLAMA = "phind/phind-codellama-34b"
MODEL_LLAMA3 = "meta-llama/llama-3-70b-instruct"
MODEL_WIZARD8x22b = "microsoft/wizardlm-2-8x22b"

# Setting up LLM

In [5]:
# Initliaze models
llm_llama3 = ChatOpenAI(openai_api_key=API_KEY, openai_api_base="https://openrouter.ai/api/v1", model_name=MODEL_LLAMA3)
llm_codellama = ChatOpenAI(openai_api_key=API_KEY, openai_api_base="https://openrouter.ai/api/v1", model_name=MODEL_CODELLAMA)
llm_wizard22 = ChatOpenAI(openai_api_key=API_KEY, openai_api_base="https://openrouter.ai/api/v1", model_name=MODEL_WIZARD8x22b)

In [6]:
def get_zip_structure(zip_file_path):
    with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
        zip_structure = io.StringIO()
        zip_ref.printdir(file=zip_structure)
        structure_text = zip_structure.getvalue().strip()
    return structure_text

In [7]:
INSTRUCT_WRAP_CODE = "\nWrap your code in ```python\n{CODE}\n```"

In [ ]:
input = "combine the images in this zip into a pdf file with one image as each page."

# input = "Convert the images in this zip file to a pdf by adding all images contained within to the PDF. Make sure to pay attention to the structure inside the zip file to make sure all code correctly handles the zipfile."

file = "images.zip"

# Generate detailed prompt from simple user prompt
template_prompt = ChatPromptTemplate.from_messages([
    ("system", """You are a helpful prompting assistant.\
    You must write instructions for creating a python script for completing the below user prompt.\
    Your job is to expand on the instructions below to make sure there is a defined plan of execution."""),
    ("user", "User Prompt: {input}\nFile: {file}\nFile Contents: {contents}")
])

template_code = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful coding assistant. Write code based on the instructions with redundancy for errors."),
    ("user", "Instructions: {input}\nFile: {file}\nFile Contents: {contents}")
])

file_contents = "\nContents of unzipped file:\n" + get_zip_structure(file)


def gen_content(llm_prompt, llm_code):
    prompt_chain = template_prompt | llm_prompt | output_parser
    
    code_chain = template_code | llm_code | output_parser
    
    prompt_output = prompt_chain.invoke({"input": input, "file": file, "contents": file_contents})
    
    prompt_output += INSTRUCT_WRAP_CODE
    
    code_output = code_chain.invoke({"input": prompt_output, "file": file, "contents": file_contents})
    
    python_code = re.search(r'```python(?s:(.*?))```', code_output).group(1)
    
    print(python_code)

gen_content(llm_llama3, llm_llama3)
gen_content(llm_codellama, llm_codellama)
gen_content(llm_wizard22, llm_wizard22)


import zipfile
from PIL import Image
from PyPDF2 import PdfFileWriter

try:
    with zipfile.ZipFile('images.zip') as zip_file:
        image_files = [f for f in zip_file.namelist() if f.endswith(('.jpg', '.jpeg'))]
        images = []
        for image_file in image_files:
            with zip_file.open(image_file) as img:
                image = Image.open(img)
                images.append(image)
except FileNotFoundError:
    print("Error: 'images.zip' file not found.")
    exit()
except zipfile.BadZipFile:
    print("Error: 'images.zip' is not a valid zip file.")
    exit()

try:
    pdf_file = PdfFileWriter()
    for image in images:
        image_page = pdf_file.addBlankPage(image.width, image.height)
        image_page.mergePage(image)
        pdf_file.addPage(image_page)
except Exception as e:
    print("Error occurred while creating PDF file:", str(e))
    exit()

try:
    with open('output.pdf', 'wb') as f:
        pdf_file.write(f)
    print("PDF file created successfully."

In [ ]:
print(prompt_output)
print(code_output)

In [14]:
import zipfile
from PIL import Image
from fpdf import FPDF
import os

try:
    # Extract the zip file
    with zipfile.ZipFile('images.zip', 'r') as zip_ref:
        zip_ref.extractall('images')
except FileNotFoundError:
    print("Error: The 'images.zip' file is not found.")
    exit()

# Create a list of image files
image_files = []
try:
    for root, dirs, files in os.walk('images'):
        for file in files:
            if file.endswith(('.jpg', '.jpeg')):
                image_files.append(os.path.join(root, file))
except FileNotFoundError:
    print("Error: The 'images' directory is not found.")
    exit()

if not image_files:
    print("Error: No image files found in the 'images' directory.")
    exit()

# Create a PDF file
pdf = FPDF()

# Add each image as a separate page
for image_file in image_files:
    try:
        image = Image.open(image_file)
        width, height = image.size
        pdf.add_page(format=(width, height))
        pdf.image(image_file, 0, 0, width, height)
    except IOError:
        print(f"Error: Unable to open image file '{image_file}'.")
        continue

# Save the PDF file
try:
    pdf.output('output.pdf', 'F')
except IOError:
    print("Error: Unable to save the PDF file.")
    exit()

print("PDF file created successfully.")

TypeError: FPDF.add_page() got an unexpected keyword argument 'format'